In [3]:
import pandas as pd
import geopandas as gpd
import os

In [4]:
# set to a local or cloud directory
data_product_dir = "/data/uscuni-ulce/data_product"

In [5]:
!ls '/data/uscuni-ulce/data_product/buildings'

at1.parquet  de2.parquet  de7.parquet  dec.parquet  lt0.parquet  pl7.parquet
at2.parquet  de3.parquet  de8.parquet  ded.parquet  pl2.parquet  pl8.parquet
at3.parquet  de4.parquet  de9.parquet  dee.parquet  pl4.parquet  pl9.parquet
cz0.parquet  de5.parquet  dea.parquet  def.parquet  pl5.parquet  sk0.parquet
de1.parquet  de6.parquet  deb.parquet  deg.parquet  pl6.parquet


In [11]:
def check_group(group):
    result = {}
    for col in attributes_to_check:
        unique_vals = group[col].dropna().unique()
        result[col] = unique_vals.tolist()
    return pd.Series(result)

In [ ]:
for filename in os.listdir("/data/uscuni-ulce/data_product/buildings"):
    buildings = gpd.read_parquet(f"{data_product_dir}/buildings/{filename}")
    morphotopes = gpd.read_parquet(f"{data_product_dir}/morphotopes/{filename}")

    # Group by morphotope_id and collect unique values for each attribute
    attributes_to_check = [
        "initially_noise",
        "level_1_label",
        "level_2_label",
        "level_3_label",
        "level_4_label",
        "level_5_label",
        "level_6_label",
        "level_7_label",
    ]

    grouped = buildings.groupby("morphotope_id").apply(check_group).reset_index()

    grouped["consistent"] = grouped.apply(
        lambda row: all(len(row[col]) == 1 for col in attributes_to_check), axis=1
    )

    inconsistent_building_groups = grouped[~grouped["consistent"]]
    print(inconsistent_building_groups)

    # Collapse lists to scalar values for consistent groups
    for col in attributes_to_check:
        grouped[col] = grouped[col].apply(lambda v: v[0] if len(v) == 1 else None)

    # Drop inconsistent ones before comparison
    grouped_consistent = grouped[grouped["consistent"]].drop(columns=["consistent"])
    morph_subset = morphotopes[["morphotope_id"] + attributes_to_check]

    # Merge on morphotope_id
    merged = grouped_consistent.merge(
        morph_subset, on="morphotope_id", suffixes=("_buildings", "_morph")
    )

    # Compare each attribute
    diffs = []
    for _, row in merged.iterrows():
        for col in attributes_to_check:
            if row[f"{col}_buildings"] != row[f"{col}_morph"]:
                diffs.append(
                    {
                        "morphotope_id": row["morphotope_id"],
                        "attribute": col,
                        "building_value": row[f"{col}_buildings"],
                        "morphotope_value": row[f"{col}_morph"],
                    }
                )

    diffs_df = pd.DataFrame(diffs)
    print(diffs_df)

In [16]:
def check_errors(filename):
    buildings = gpd.read_parquet(f"{data_product_dir}/buildings/{filename}")
    morphotopes = gpd.read_parquet(f"{data_product_dir}/morphotopes/{filename}")

    # Group by morphotope_id and collect unique values for each attribute
    attributes_to_check = [
        "initially_noise",
        "level_1_label",
        "level_2_label",
        "level_3_label",
        "level_4_label",
        "level_5_label",
        "level_6_label",
        "level_7_label",
    ]

    def check_group(group):
        result = {}
        for col in attributes_to_check:
            unique_vals = group[col].dropna().unique()
            result[col] = unique_vals.tolist()
        return pd.Series(result)

    grouped = buildings.groupby("morphotope_id").apply(check_group).reset_index()

    grouped["consistent"] = grouped.apply(
        lambda row: all(len(row[col]) == 1 for col in attributes_to_check), axis=1
    )

    inconsistent_building_groups = grouped[~grouped["consistent"]]
    # Collapse lists to scalar values for consistent groups
    for col in attributes_to_check:
        grouped[col] = grouped[col].apply(lambda v: v[0] if len(v) == 1 else None)

    # Drop inconsistent ones before comparison
    grouped_consistent = grouped[grouped["consistent"]].drop(columns=["consistent"])
    morph_subset = morphotopes[["morphotope_id"] + attributes_to_check]

    # Merge on morphotope_id
    merged = grouped_consistent.merge(
        morph_subset, on="morphotope_id", suffixes=("_buildings", "_morph")
    )

    # Compare each attribute
    diffs = []
    for _, row in merged.iterrows():
        for col in attributes_to_check:
            if row[f"{col}_buildings"] != row[f"{col}_morph"]:
                diffs.append(
                    {
                        "morphotope_id": row["morphotope_id"],
                        "attribute": col,
                        "building_value": row[f"{col}_buildings"],
                        "morphotope_value": row[f"{col}_morph"],
                    }
                )

    diffs_df = pd.DataFrame(diffs)
    print(filename)
    print(diffs_df)

In [17]:
for filename in os.listdir("/data/uscuni-ulce/data_product/buildings"):
    check_errors(filename)

/tmp/ipykernel_266956/1764903699.py:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = buildings.groupby("morphotope_id").apply(check_group).reset_index()


deb.parquet
    morphotope_id      attribute  building_value  morphotope_value
0    10095_63_-92  level_2_label             2.0                 1
1    10095_63_-92  level_3_label             4.0                 2
2    10095_63_-92  level_4_label             6.0                 3
3    10095_63_-92  level_5_label             9.0                 4
4    10095_63_-92  level_6_label            18.0                 8
5    10095_63_-92  level_7_label            36.0                14
6   3109_840_-805  level_2_label             2.0                 1
7   3109_840_-805  level_3_label             4.0                 2
8   3109_840_-805  level_4_label             6.0                 3
9   3109_840_-805  level_5_label             9.0                 4
10  3109_840_-805  level_6_label            19.0                 8
11  3109_840_-805  level_7_label            38.0                15
12    3221_52_-99  level_2_label             2.0                 1
13    3221_52_-99  level_3_label             5.0  

/tmp/ipykernel_266956/1764903699.py:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = buildings.groupby("morphotope_id").apply(check_group).reset_index()


ded.parquet
    morphotope_id      attribute  building_value  morphotope_value
0  53380_499_-696  level_4_label             5.0                 6
1  53380_499_-696  level_5_label             8.0                 9
2  53380_499_-696  level_6_label            17.0                18
3  53380_499_-696  level_7_label            35.0                36
4  68363_193_-205  level_6_label            18.0                19
5  68363_193_-205  level_7_label            36.0                38


/tmp/ipykernel_266956/1764903699.py:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = buildings.groupby("morphotope_id").apply(check_group).reset_index()


...

at3.parquet
         morphotope_id      attribute  building_value  morphotope_value
0         29374_29_-30  level_4_label               5                 6
1         29374_29_-30  level_5_label               8                 9
2         29374_29_-30  level_6_label              17                18
3         29374_29_-30  level_7_label              35                36
4       29374_332_-340  level_4_label               6                 5
5       29374_332_-340  level_5_label               9                 8
6       29374_332_-340  level_6_label              18                17
7       29374_332_-340  level_7_label              36                35
8       35141_287_-305  level_4_label               6                 5
9       35141_287_-305  level_5_label               9                 8
10      35141_287_-305  level_6_label              19                17
11      35141_287_-305  level_7_label              38                35
12    50280_2440_-2451  level_7_label              2

/tmp/ipykernel_266956/1764903699.py:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = buildings.groupby("morphotope_id").apply(check_group).reset_index()


de9.parquet
      morphotope_id      attribute  building_value  morphotope_value
0    15794_386_-377  level_6_label            15.0                13
1    15794_386_-377  level_7_label            32.0                26
2     2096_462_-495  level_5_label             6.0                 7
3     2096_462_-495  level_6_label            12.0                13
4     2096_462_-495  level_7_label            23.0                28
5   20970_753_-1239  level_4_label             6.0                 5
6   20970_753_-1239  level_5_label             9.0                 8
7   20970_753_-1239  level_6_label            18.0                17
8   20970_753_-1239  level_7_label            36.0                35
9     3705_276_-308  level_5_label            19.0                18
10    3705_276_-308  level_6_label            45.0                40
11    3705_276_-308  level_7_label            82.0                74
12  4214_5612_-6346  level_5_label            10.0                 9
13  4214_5612_-6346  l

/tmp/ipykernel_266956/1764903699.py:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = buildings.groupby("morphotope_id").apply(check_group).reset_index()


de8.parquet
    morphotope_id      attribute  building_value  morphotope_value
0  31212_246_-221  level_3_label               3                 4
1  31212_246_-221  level_4_label               4                 5
2  31212_246_-221  level_5_label               7                 8
3  31212_246_-221  level_6_label              13                17
4  31212_246_-221  level_7_label              28                35
5   38243_36_-497  level_5_label              10                 9
6   38243_36_-497  level_6_label              21                18
7   38243_36_-497  level_7_label              44                36
